# Design of a Balance of Plant System with Dynamic Networks
> Nik Kemper, Jack Kennedy and Valmor F. de Almeida <br>
> *nikolaus\_kemper@student.uml.edu, jack_kennedy@student.uml.edu and valmor\_dealmeida@uml.edu*<br>
>
> **University of Massachusetts Lowell** <br>
> Dept. of Chemical Engineering, Nuclear Energy Program, Lowell, MA, 01854, <br>
>
> **2021 Virtual SciPy Poster Session**<br>
> Tuesday, July 13, 2021 (2:15–4:00PM EDT) <br>
> Austin, TX (online)
$  
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\Smtrx}{\boldsymbol{\mathsf{S}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\uvec}{\boldsymbol{u}}
  \newcommand{\pvec}{\boldsymbol{p}}
  \newcommand{\qvec}{\boldsymbol{q}}
  \newcommand{\uvar}{\boldsymbol{u}}
  \newcommand{\fvar}{\boldsymbol{f}}
  \newcommand{\Fvar}{\boldsymbol{F}}
  \newcommand{\Gvar}{\boldsymbol{G}}
  \newcommand{\Hvar}{\boldsymbol{H}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\mvec}{\boldsymbol{\mathsf{m}}}
  \newcommand{\gvec}{\boldsymbol{\mathsf{g}}}
  \newcommand{\zerovec}{\boldsymbol{\mathsf{0}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \newcommand{\transpose}[1]{{#1}^\top}
  \DeclareMathOperator{\rank}{rank}
  \newcommand{\Power}{\mathcal{P}}
  \DeclareMathOperator{\dt}{d_\textit{t}}
$

---
### <center> Bringing high-performance, system-level, dynamic modeling and simulation to the Python ecosystem: </center>
### <center> "How to get there without Symulink while scaling to high-perfomance computing platforms" </center>
---

<center> 
<img width="600" src="figs/network.png" title="Plant Layout">
<p style="text-align:center;"><b>Balance of plant network graph (SMPWR: Small Modular Pressurized Water Reactor).</b></p> 
</center>

---
## Summary<a id="summary"></a>
We created an extended Balance of Plant (BoP) simulation network using the [`Cortix`](https://cortix.org) Python library. The BoP system was made by connecting multiple `Cortix` modules that modeled unit operations (such as steam
generators, turbines, water heaters, *etc.*) as a networked system. Modules were connected via `Cortix` *ports* that are designated to either
send or receive parallel, message-passing information from other modules. The information exchanged consisted of data
structures representing either flowing water streams that contained temperature, pressure, and flowrate data or energy
streams that transferred energy. Simulations of start up, steady state, shutdown and step change conditions were
performed.

---

## Abstract<a id="abstract"></a>

The nuclear power industry has a long history in using modeling and simulation to design nuclear systems. Both
experience and existing simulation software can be leveraged to build modeling capabilities to extended balance of
plants (BoPs). Here we adopt a divide-and-conquer approach for modeling and simulation wherein significant sub-systems are modeled individually and coupled via an external framework. 

A power plant has many sub-systems that appear repeatedly, either physically or logically, say a feed water system composed of many individual water heaters, or a Rankine turbine with many regenerative heating steps. In both cases, a network can be built for all the components, and a dynamic model assembled to represent an extended BoP. A key capability requirement of the
framework is to allow for scalability and reusability of computational modules. Given the current status of modern
computing power, extended BoP’s can now be created to various levels of detail; for instance, valves and pipes can
be included explicitly in the network. 

An extended network can be mapped on large scale parallel computing
platforms to allow for increasingly modeling fidelity. This forward picture calls for a broader understanding of the
elements of a power plant and the coupling of the corresponding flow of energy. This is the primary motivation of
this project.

We developed and used five system-level modules to represent a nuclear facility. These modules included a reactor,
a steam generator, a turbine, a condenser, and a feedwater heating system. The primary loop flows between the
reactor module and the steam generator, while the secondary loop flows from the water heater, to the steam
generator, to the turbine, to the condenser, where it then flows back to the water heater. 

The `Cortix` Python library
was used to connect each of the modules. This was done by connecting information streams through inflow and
outflow *ports* on each of the modules. We also created an accident scenario where the secondary loop’s mass
flowrate was temporarily reduced so we could see the effect it would have on our system.

The [`IAPWS`](https://github.com/jjgomera/iapws) Python library was fully coupled to systematically compute the thermodynamic and physical properties of water and steam at many different conditions. By coupling the system level models through [Cortix](https://cortix.org), and using [`SciPy`](https://www.scipy.org) mathematical modules (ODE
integrator) we were able to create an extended time-dependent BoP system which can be used to examine and model
how startup, shutdown, and loss of coolant accidents impact each part of the plant.


## [Cortix framework for system-level, large scale, network modeling and simulation](#toc)<a id="cortix"></a>
1. Framework for developing system-level applications
 - All-`python`, non-commercial free,  open-source [(download site)](https://cortix.org) development
1. Applications consist of a generic *bi-directional network* of stand-alone *computational modules*
 - Collection of coupled, reduced-order models
1. `Cortix` provides a single `Module` *class* for implementation of computational modules
 - Wrapper of the developer's computational module
 - Imposes *light* restrictions on the architecture of an existing code to allow for network coupling
1. `Module` is implemented using existing message passing parallel communication libraries for data coupling.
 - Module developer does not need *in-depth* understanding of message passing libraries
 - The understanding of a `Port` is required for effective communication programming
1. `Cortix` handles the connectivity of the network and delivery/receipt of messages
 - Messages can be any serializable form of data structure (*e.g.* time-stamped, events, etc.)
 - Framework is *hands-off* data; module developer is free to use any data structure
1. Framework assigns a process to each `Module` in the network
 - Application runs in parallel: either processes (MPI, `mpi4py`) or threads (`Python` `multiprocessing`)
 - Tested in HPC platforms, laptops, desktops, and the cloud: Linux, Mac, Windows (Anaconda/VM)
 - Scalability performance dependent on computational modules (`Network` and `Port` management is scalable)
1. `Cortix` is aimed at the HPC platform and uses the <span style="color:red">rich `Python` ecosystem</span>; 
 - <span style="color:red">Fully integrated into electronic `Jupyter` notebooks</span>: teaching, collaboration, development, and research

## [Divide and Couple Approach to Complex Systems](#toc)<a id="cortix2"></a>
An example of time-dependent system of coupled modules with indicated flow of information
<center>
<img width="500" src="figs/many-m.png" title="Module Network">
</center>

+ $M_i$: module of a reduced-order model
+ $\qvec_{\vec{i,j}}(t)$: coupling data vector (unknowns, and <span style="color:red">signals</span>)
+ $\pvec_{i}(\qvec_i, t)$: parameter vector (known function of time or unknowns)
<br>

Sources of mathematical computational and complexity:
 + Large scale connectivity (scaling up to realistic sizes)
 + Module behavior dependent on signals (on/off, breakdown, warming up, cooling down, *etc.*)
 
NB: Since `Cortix` is designed on the basis of *messages*, event-driven systems are equally supported.

## [Module Internals and Commonality](#toc)<a id="cortix2"></a>
Reduced-order models often share a common mathematical basis because of balance laws, *e.g.*

|  | | |
|---|--|---|
|<img width="300" src="figs/black-box.png" title="Module"> | >> INTERNALLY >> | <img width="600" src="figs/white-box.png" title="Module">|

+ $\Fvar(\cdot), \Gvar(\cdot), \Hvar(\cdot)$: behavior/model
+ $\uvec(t)$: state vector
+ $\pvec(t)$: parameter vector
+ $\qvec(t)$: coupling vector (depends on a subset of the state vector)

Often, the model is a system of non-linearly coupled ODEs.

## [Life of a Cortix Module](#toc)<a id="cortix2"></a>
<center>
<img width="300" src="figs/external-behavior.png" title="Module">
</center>

+ A module $M_i$ does not receive or send a message to itself
+ $M_i$ may evolve on its own time $t_i$
  - Requires a known parameter vector $\pvec_i(t_i)$
+ Coupling vectors $\qvec_{\vec{j, i}}(t_i)$ allow $M_i$ to use data from other modules in the network.
  - $M_i$ will *wait* for time-stamped coupling data $t_j$ via message passing (synchronization)

## [Time Evolution (uniform case)](#toc)<a id="cortix2"></a>

1. Initialize $t_0  = 0$ and $M_j\bigl(\uvec(t_0); \qvec(t_0), \pvec(t_0)\bigr)$ for all modules in the network.
1. For all $i = 1 \ldots n$:
 + Solve for $\uvec(t_i)$  $\forall$  $M_j\bigl(\uvec(t_i); \qvec(t_{i - 1}), \pvec(t_{i - 1})\bigr)$ independently.
 + Compute $\qvec(t_i), \pvec(t_i)$ and exchange information: $\qvec_{\vec{j, k}}(t_i)$.
 + Advance $t_{i + 1} \leftarrow t_i + \Delta t$ according to the configured time step.

### Comments
+ There is inevitable propagation of error in this explicit time evolution.
+ Modules must be loosely coupled components of the system; merge those that are tightly coupled into one.
+ Reduce the integration time step to the extent possible.

## [Modular Network Assembly](#toc)<a id="cortix2"></a>
+ Balance of plant network

In [ ]:
'''Balance of Plant'''
# Framework
from cortix import Cortix
from cortix import Network

plant = Cortix()                      # System top level
plant_net = plant.network = Network() # Network

In [ ]:
'''Developer Reactor (Small Modular Pressurized Water Reactor)'''
from bop.reactor import SMPWR

reactor = SMPWR()     # Create reactor module

plant_net.module(reactor) # Add reactor module to network

In [ ]:
'''Developer Steamer'''
from bop.steamer import Steamer

steamer = Steamer() # Create turbine
steamer.name = 'Steamer'

plant_net.module(steamer) # Add turbine module to network

In [ ]:
'''Developer Turbine'''
from bop.turbine import Turbine
turbine = Turbine()  # Create turbine module

plant_net.module(turbine)  # Add turbine module to network

In [ ]:
'''Developer Condenser'''
from bop.condenser import Condenser
condenser = Condenser()  # Create condenser module

plant_net.module(condenser)  # Add condenser module to network

In [ ]:
'''Developer Water Heater'''
from bop.water_heater import WaterHeater
water_heater = WaterHeater()  # Create water_heater module

plant_net.module(water_heater)  # Add water_heater module to network

### Exchange of Information $\qvec_{\vec{j, k}}(t_i)$:
+ `Ports` are the mechanism for exchanging information.
+ `Ports` are provided in the `Module` class from `Cortix`.
+ Either the MPI or `Python` `multiprocessing` libraries are used.

In [ ]:
'''Balance of Plant Network Connectivity'''
# Connect an edge or pipe (ports are the ends of the edge)

plant_net.connect([reactor, 'coolant-outflow'], [steamer, 'primary-inflow'])
plant_net.connect([steamer, 'primary-outflow'], [reactor, 'coolant-inflow'])
plant_net.connect([steamer, 'secondary-outflow'], [turbine, 'inflow'])
plant_net.connect([turbine, 'outflow'], [condenser, 'inflow'])
plant_net.connect([turbine, 'process-heat'], [water_heater, 'external-heat'])
plant_net.connect([condenser, 'outflow'], [water_heater, 'inflow'])
plant_net.connect([water_heater, 'outflow'], [steamer, 'secondary-inflow'])

In [ ]:
'''Draw network'''
plant_net.draw(engine='circo', node_shape='folder')

### Comments:
+ Note general bi-directional graph connectivity
+ Note that there is no parsing of an input file to setup system.
+ `Python` programming language is used to create the configuration information directly.

## [Scalability of Assembly (Ports and Pipes)](#toc)<a id="cortix2"></a>
+ Ports are *cheap* to create (*i.e* little overhead) and can be created on the fly.
+ For example (assembly of droplets immersed into a vortex):

In [ ]:
from cortix.examples.droplet_swirl.droplet import Droplet
from cortix.examples.droplet_swirl.vortex import Vortex

# Configuration Parameters
n_droplets = 10
  
swirl = Cortix()
swirl.network = Network()
  
# Vortex module (single).
vortex = Vortex()
swirl.network.module(vortex)

for i in range(n_droplets):
# Droplet modules (multiple).
    droplet = Droplet()
    droplet.name = 'Droplet %i'%(i+1)
    swirl.network.module(droplet)
  
# Network port connectivity (connect modules through their ports)
    swirl.network.connect([droplet, 'external-flow'],
                          [vortex, vortex.get_port('fluid-flow:{}'.format(i))], 'bidirectional')
  
swirl.network.draw(engine='twopi')

In [ ]:
swirl.close()

### Comment
+ In cases with module repeatability the assembly process can be easily automated (as demonstrated).

## [Balance of Plant Problem Statement](#toc)<a id="ps"></a>

<span style="color:red">THIS NEEDS UPDATE</span>

Steam is created from nuclear heat generated in the reactor and passed to the high-pressure turbine, which expands the steam to about 0.5 MPA (or 5 bar). The runoff from the high-pressure turbine is then split in half and fed to the two low-pressure turbines, which expand the runoff to 0.5 bar. The runoff is then fed to a condenser which produces subcooled liquid water, which is then fed back into the bottom of the reactor.

Modeling the entire network is accomplished through the use of the network simulation library [Cortix.](https://cortix.org). The individual plant components (reactor, turbines and condenser) are implemented as `Cortix` modules, with data exchange handled by the Cortix library.

Additionally, a number of the calculations used in this model rely on the [IAPWS](https://pypi.org/project/iapws/) library developed by Juan José Gómez Romera (jjgomera). IAPWS is the International Association for the Properties of Water and Steam, which develops power series that can be used to calculate properties of water and steam (such as specific enthalpy and entropy) at a variety of temperatures, pressures and qualities. The IAPWS library devloped by jjgomera implements these series as python functions which can be easily accessed and called to calculate any property required for thermodynamic calculations, drastically simplifying the process of modelling the two-phase systems in use by the turbine and condenser modules. This project specifically uses the IAPWS97 module within the library, which uses the formulations published in 1997 by IAPWS foundation as an update to ones developed in 1967. 

## [Module Basic Template for the Run Method](#toc)<a id="moduletemplate"></a>

```python
class BWR(Module): # inherit from the base class Module

    def run(self, *args):
          # Module's time loop
          while time <= self.end_time:
              # Specific coding goes here (operation, on-off, status, etc.)

              # Evolve one time step
              #---------------------
              time = self.__step(time)
        
              # Communicate information
              #------------------------
              self.__call_ports(time)
                
    def __step(self, time):
        # 1. Prepare module data for ODE solver        
        # 2. Call ODE solver
        # 3. Advance time
        time += self.time_step
        # 4. Update internal state data
        return time
    
     def __call_ports(self, time):

         # Interactions in the coolant-outflow port
         #-----------------------------------------
         # One way "to" coolant-outflow
  
         # Send to
         if self.get_port('coolant-outflow').connected_port:
  
              msg_time = self.recv('coolant-outflow')

              temp = self.coolant_outflow_phase.get_value('temp', msg_time)
              press = self.coolant_outflow_phase.get_value('pressure', msg_time)
  
              coolant_outflow = dict()
              coolant_outflow['temperature'] = temp
              coolant_outflow['pressure'] = press
  
              self.send((msg_time, coolant_outflow), 'coolant-outflow')
  
          # Interactions in the coolant-inflow port
          #----------------------------------------
          # One way "from" coolant-inflow
  
          # Receive from
          if self.get_port('coolant-inflow').connected_port:
  
              self.send(time, 'coolant-inflow')
  
              inflow_coolant = self.recv('coolant-inflow')
  
              self.coolant_mass_flowrate = inflow_coolant['mass_flowrate']
              self.coolant_quality = inflow_coolant['quality']
```

## [Brief Description of BOP Modules](#toc)<a id="bwr"></a>

In [ ]:
plant_net.draw(engine='circo', node_shape='folder')

---
## [Single-Point Pressurized Water Reactor Model](#toc)<a id="bwr"></a>

<span style="color:red">THIS NEEDS UPDATE</span>


---
## [Turbine Model](#toc) <a id="tub"></a>

<span style="color:red">THIS NEEDS UPDATE</span>


---
## [Condenser Model](#toc) <a id="cond"></a>

<span style="color:red">THIS NEEDS UPDATE</span>


---
## [Water Heater](#toc) <a id="cond"></a>

<span style="color:red">THIS NEEDS UPDATE</span>


## [Network Simulation Results](#toc) <a id="cortix"></a>

## [Startup](#toc)<a id="su"></a>

Here the simulation runs for a period representative of a startup interval.

In [ ]:
'''Run BOP Network'''

from bop.support import unit

end_time = 10*unit.minute 
time_step = 1.5*unit.second
show_time = (True, 5*unit.minute)

for module in plant_net.modules:
    module.time_step = time_step
    module.end_time = end_time
    module.show_time = show_time

plant.run()   # Run network dynamics simulation
plant.close() # Close run

In [ ]:
'''Collect multiprocessing data'''
reactor = plant_net.modules[0]
steamer = plant_net.modules[1]
turbine = plant_net.modules[2]
condenser = plant_net.modules[3]
water_heater = plant_net.modules[4]

In [ ]:
'''Reactor data'''
reactor.state_phase.plot(actors=['power','inlet-temp'], show=True, figsize=[14,8])
reactor.coolant_outflow_phase.plot(actors=['temp','flowrate'], show=True, figsize=[14,8])

In [ ]:
'''Steamer data'''
steamer.primary_outflow_phase.plot(show=True, figsize=[14,8])
steamer.secondary_outflow_phase.plot(actors=['temp','quality'], show=True, figsize=[14,8])
steamer.state_phase.plot(actors=['heatflux'], show=True, figsize=[14,8])

## [Steady State](#toc)<a id="ss"></a>

The previous startup simulation can be continued until a steady state is achieved.

In [ ]:
'''Continue simulation to steady state'''

end_time += 10*unit.minute

for module in plant_net.modules:
    module.initial_time = module.end_time
    module.end_time = end_time

# Rebuild the network since this information is lost once the Cortix object is closed
plant_net.connect([reactor, 'coolant-outflow'], [steamer, 'primary-inflow'])
plant_net.connect([steamer, 'primary-outflow'], [reactor, 'coolant-inflow'])
plant_net.connect([steamer, 'secondary-outflow'], [turbine, 'inflow'])
plant_net.connect([turbine, 'outflow'], [condenser, 'inflow'])
plant_net.connect([turbine, 'process-heat'], [water_heater, 'external-heat'])
plant_net.connect([condenser, 'outflow'], [water_heater, 'inflow'])
plant_net.connect([water_heater, 'outflow'], [steamer, 'secondary-inflow'])

In [ ]:
plant.run()   # Run network dynamics simulation
plant.close() # Close run

In [ ]:
'''Collect multiprocessing data'''
reactor = plant_net.modules[0]
steamer = plant_net.modules[1]
turbine = plant_net.modules[2]
condenser = plant_net.modules[3]
water_heater = plant_net.modules[4]

In [ ]:
'''Reactor data'''
reactor.state_phase.plot(actors=['power','inlet-temp'], show=True, figsize=[14,8])
reactor.coolant_outflow_phase.plot(actors=['temp','flowrate'], show=True, figsize=[14,8])

In [ ]:
'''Turbine data'''
turbine.state_phase.plot(show=True, figsize=[14,8])
turbine.outflow_phase.plot(actors=['temp','quality'], show=True,figsize=[14,8])

## [Water Heater Malfunction](#toc)<a id="malfunction"></a>

Continuing the simulation from the steady state above, a malfunction is created by interrupting the line from the water heater module to the steamer module for a given interval of time; normal conditions applied when the interruption ends.

In [ ]:
'''Continue simulation with malfunction of water heater'''

end_time += 25*unit.minute

for module in plant_net.modules:
    module.initial_time = module.end_time
    module.end_time = end_time
    
plant_net.connect([reactor, 'coolant-outflow'], [steamer, 'primary-inflow'])
plant_net.connect([steamer, 'primary-outflow'], [reactor, 'coolant-inflow'])
plant_net.connect([steamer, 'secondary-outflow'], [turbine, 'inflow'])
plant_net.connect([turbine, 'outflow'], [condenser, 'inflow'])
plant_net.connect([turbine, 'process-heat'], [water_heater, 'external-heat'])
plant_net.connect([condenser, 'outflow'], [water_heater, 'inflow'])
plant_net.connect([water_heater, 'outflow'], [steamer, 'secondary-inflow'])

In [ ]:
initial_malfunction_time = water_heater.initial_time + 5*unit.minute
final_malfunction_time = initial_malfunction_time + 15*unit.minute

water_heater.malfunction = (True, initial_malfunction_time, final_malfunction_time)

plant.run()   # Run network dynamics simulation
plant.close() # Close run

In [ ]:
'''Collect multiprocessing data'''
reactor = plant_net.modules[0]
steamer = plant_net.modules[1]
turbine = plant_net.modules[2]
condenser = plant_net.modules[3]
water_heater = plant_net.modules[4]

In [ ]:
'''Reactor data'''
reactor.state_phase.plot(actors=['power','inlet-temp'], show=True,figsize=[14,8])
reactor.coolant_outflow_phase.plot(actors=['temp','flowrate'], show=True,figsize=[14,8])

In [ ]:
'''Turbine data'''
turbine.state_phase.plot(show=True, figsize=[14,8])
turbine.outflow_phase.plot(actors=['temp','quality'], show=True,figsize=[14,8])

In [ ]:
'''Water heater data'''
water_heater.inflow_phase.plot(actors=['external-heat'], show=True,figsize=[14,8])
water_heater.outflow_phase.plot(actors=['temp','flowrate'], show=True,figsize=[14,8])

In [ ]:
'''Steamer data'''
steamer.primary_outflow_phase.plot(show=True, figsize=[14,8])
steamer.secondary_outflow_phase.plot(actors=['temp','quality'], show=True, figsize=[14,8])
steamer.state_phase.plot(actors=['heatflux'], show=True, figsize=[14,8])

## [Shutdown](#toc)<a id="sd"></a>
Finally, continuing the simulation from the normal conditions after the malfunction above, the simulation is told to shutdown the plant.

In [ ]:
'''Continue simulation with malfunction of water heater'''

end_time += 15*unit.minute

for module in plant_net.modules:
    module.initial_time = module.end_time
    module.end_time = end_time
    
plant_net.connect([reactor, 'coolant-outflow'], [steamer, 'primary-inflow'])
plant_net.connect([steamer, 'primary-outflow'], [reactor, 'coolant-inflow'])
plant_net.connect([steamer, 'secondary-outflow'], [turbine, 'inflow'])
plant_net.connect([turbine, 'outflow'], [condenser, 'inflow'])
plant_net.connect([turbine, 'process-heat'], [water_heater, 'external-heat'])
plant_net.connect([condenser, 'outflow'], [water_heater, 'inflow'])
plant_net.connect([water_heater, 'outflow'], [steamer, 'secondary-inflow'])

In [ ]:
shutdown_time = reactor.initial_time + 5*unit.minute

reactor.shutdown = (True, shutdown_time)

plant.run()   # Run network dynamics simulation
plant.close() # Close run

In [ ]:
'''Collect multiprocessing data'''

reactor = plant_net.modules[0]
steamer = plant_net.modules[1]
turbine = plant_net.modules[2]
condenser = plant_net.modules[3]
water_heater = plant_net.modules[4]

In [ ]:
'''Reactor data'''
#reactor.state_phase.plot(show=True, figsize=[14,8])
reactor.state_phase.plot(actors=['power','inlet-temp'], show=True,figsize=[14,8])
reactor.coolant_outflow_phase.plot(actors=['temp','flowrate'], show=True,figsize=[14,8])
reactor.neutron_phase.plot(actors=['neutron-dens'], show=True, figsize=[14,8])

In [ ]:
'''Turbine data'''
turbine.state_phase.plot(show=True, figsize=[14,8])
turbine.outflow_phase.plot(actors=['temp','quality'], show=True,figsize=[14,8])

In [ ]:
'''Steamer data'''
steamer.primary_outflow_phase.plot(show=True, figsize=[14,8])
steamer.secondary_outflow_phase.plot(actors=['temp','quality'], show=True, figsize=[14,8])
steamer.state_phase.plot(actors=['heatflux'], show=True, figsize=[14,8])

In [ ]:
'''Water heater data'''
water_heater.inflow_phase.plot(actors=['external-heat'], show=True,figsize=[14,8])
water_heater.outflow_phase.plot(actors=['temp','flowrate'], show=True,figsize=[14,8])

## [Final Comments](#toc)<a id="finalcomments"></a>

+ System-level models can be coupled and assembled into a dynamic network using `Cortix` and the Python ecosystem.
+ Using message passing and parallel processing supported by `Cortix`, a simulation can be executed in various computing platforms including high-performance computers.
+ Full integration to Jupyter notebooks accelerates collaboration and programming.


### Thank you for stopping by.

## [Acknowledgements](#toc)<a id="ackn"></a>

This work was partially funded by the [Francis College of Engineering at UMass Lowell](https://www.uml.edu/engineering/), Department of Chemical Engineering (Nuclear Program), and the [Cortix](https://cortix.org) group.